In [ ]:
import pandas as pd
import re
import math

In [ ]:
file_dir='Copy of Riset Pasar Kaki Keenam (Responses).xlsx'

In [ ]:
#Kasus spesial yg rada malesin kalo dibuat versi regexnya. Diganti saat dipecah per entry
#specialcasefood={'cilok':'aci','cimol':'aci','cilor':'aci','cireng':'aci','cilung':'aci','cakwe':'gorengan lain','basreng':'gorengan lain','sate kikil':'sate','soto':'soto/sop/makanan berkuah lainnya','kebab turki':'kebab'}
#array buat kasus khusus yg ribet diganti pake regex
specialcasefood={'soto':'soto/sop/makanan berkuah lainnya'}
specialcasedrink={}
stupidcase=['etc','random','','wedang','cincau']

In [ ]:
food_threshold=3
drink_threshold=2

In [ ]:
data=pd.read_excel(file_dir)

In [ ]:
data

In [ ]:
#ambil baris data makanan
foodarr=data['Kalau jajan di pedagang kecil, Kamu sukanya jajanan makanan apa? (Bisa isi lebih dari satu)'].str.lower().str.strip(' ')
#regexing all the stupid entry
foodarr=foodarr.replace(r"ci[a-z]*","gorengan kering",regex=True)
foodarr=foodarr.replace(r"(cakwe|basreng)","gorengan kering",regex=True)
foodarr=foodarr.replace(r"sate[a-z ]*","sate",regex=True)
#foodarr=foodarr.replace(r"(soto [a-z ]*|sop [a-z ]*)","soto/sop/makanan berkuah lainnya",regex=True)
foodarr=foodarr.replace(r"kebab[a-z ]*","kebab",regex=True)
foodarr=foodarr.replace(r"soto$","soto/sop/makanan berkuah lainnya",regex=True)
foodarr=foodarr.replace(r"bala bala","gorengan kering",regex=True)
foodarr=foodarr.replace(r"lotek","bakso/batagor/siomay",regex=True)
#burger, dimsum, kebab, macaroni, sosis, takoyaki = makanan luar negeri
foodarr=foodarr.replace(r"burger","luar negeri",regex=True)
foodarr=foodarr.replace(r"dimsum","luar negeri",regex=True)
foodarr=foodarr.replace(r"kebab","luar negeri",regex=True)
foodarr=foodarr.replace(r"macaroni","luar negeri",regex=True)
foodarr=foodarr.replace(r"sosis","luar negeri",regex=True)
foodarr=foodarr.replace(r"takoyaki","luar negeri",regex=True)
#lumpia basah, seblak = gorengan basah
foodarr=foodarr.replace(r"lumpia basah","gorengan basah",regex=True)
foodarr=foodarr.replace(r"seblak","gorengan basah",regex=True)
foodarr=foodarr.replace(r"ayam bakar/goreng/fried chicken","ayam",regex=True)
foodarr=foodarr.replace(r"bakso/batagor/siomay","bakso",regex=True)
foodarr=foodarr.replace(r"buah potong/rujak","buah",regex=True)
foodarr=foodarr.replace(r"gorengan lain","gorengan kering",regex=True)
foodarr=foodarr.replace(r"kue-kue","kue",regex=True)
foodarr=foodarr.replace(r"mie-miean","mie",regex=True)
foodarr=foodarr.replace(r"nasi-nasian","nasi",regex=True)
foodarr=foodarr.replace(r"roti bakar/kukus","roti",regex=True)
foodarr=foodarr.replace(r"martabak","kue",regex=True)
foodarr=foodarr.replace(r"asinan","buah",regex=True)
foodarr=foodarr.replace(r"soto/sop/makanan berkuah lainnya","berkuah",regex=True)

In [ ]:
#foodarr.unique()

In [ ]:
#menghitung total makanan dari kuesioner ke dictionary
fooddict={}
z=0
for i in foodarr:
    #biar engga duplikat
    temp=dict.fromkeys(i.split(', '))
    if len(temp) < 3:
      continue
    z+=1
    #split karena data terpisah oleh koma
    for j in temp:
        if j in stupidcase:
            continue
        if j not in specialcasefood:
            if j in fooddict:
                fooddict[j]=fooddict[j]+1
            else:
                fooddict[j]=1
        else:
            entry=specialcasefood[j]
            if entry in fooddict:
                fooddict[entry]=fooddict[entry]+1
            else:
                fooddict[entry]=1
print(z)

In [ ]:
#hasil disortir dari jumlah terbanyak
dict(sorted(fooddict.items(), key=lambda item: item[1]))

In [ ]:
#aci, bala, = gorengan lain
#fooddict['gorengan lain']+=fooddict.pop("aci")
#fooddict['gorengan lain']+=fooddict.pop("bala bala")
#dimsum,lotek = bakso/batagor/sioimay
#fooddict['bakso/batagor/siomay']+=fooddict.pop("dimsum")
#fooddict['bakso/batagor/siomay']+=fooddict.pop("lotek")
#burger, dimsum, kebab, macaroni, sosis, takoyaki = makanan luar negeri
#lumpia basah, seblak = gorengan basah
#fooddict['gorengan basah']=7
#fooddict['luar negeri']+=1
#fooddict['bakso/batagor/siomay']-=1

In [ ]:
#fooddict['ayam']=fooddict.pop('ayam bakar/goreng/fried chicken')
#fooddict['bakso']=fooddict.pop('bakso/batagor/siomay')
#fooddict['buah']=fooddict.pop('buah potong/rujak')#Buah potong bla bla = Buah
#fooddict['gorengan kering']=fooddict.pop('gorengan lain')#gorengan lain = gorengan kering
#fooddict['kue']=fooddict.pop("kue-kue")# = kue
#fooddict['mie']=fooddict.pop("mie-miean")# = mie
#fooddict['nasi']=fooddict.pop("nasi-nasian")#  = nasi
#fooddict['roti']=fooddict.pop("roti bakar/kukus")# = roti
#fooddict['berkuah']=fooddict.pop("soto/sop/makanan berkuah lainnya")#soto/sop = berkuah
dict(sorted(fooddict.items(), key=lambda item: item[1]))

In [ ]:
#buat kategori makanan yang terindex
food_cat=pd.CategoricalIndex(fooddict)
#ubah makanan dari nama ke angka
hotencoded_foodarr=[]
for i in foodarr:
    temparr=[]
    temp=dict.fromkeys(i.split(', '))
    if len(temp) < food_threshold:
        hotencoded_foodarr.append([])
        continue
    for j in food_cat:
        if j in i.split(', '):
            temparr.append(1)
        else:
            temparr.append(0)
    hotencoded_foodarr.append(temparr)

MINUMAN

In [ ]:
#ambil baris data makanan
drinkarr=data['Kalau jajan di pedagang kecil, Kamu sukanya minuman apa? (Bisa isi lebih dari satu)'].str.lower().str.strip(' ')
#drinkarr.unique()
#cleaning all the different entry
drinkarr=drinkarr.replace(r"wedang[a-z ]*","wedang",regex=True)
drinkarr=drinkarr.replace(r"(air[a-z ]*|mineral[a-z ]*|aqua[a-z ]*)","air mineral",regex=True)
drinkarr=drinkarr.replace(r"([a-z][a-z ]*latte[a-z ]*|capucino[a-z ]*)","kopi",regex=True)
drinkarr=drinkarr.replace(r"[a-z][a-z ]*jeruk","jus buah",regex=True)
drinkarr=drinkarr.replace(r"teh[a-z ]*","teh",regex=True)
drinkarr=drinkarr.replace(r"(pop ice|minuman saset)","soda/soft drink",regex=True)
drinkarr=drinkarr.replace(r"es boba","bubble tea",regex=True)
drinkarr=drinkarr.replace(r"susu[a-z ]*","susu",regex=True)
drinkarr=drinkarr.replace(r"(es degan[a-z ]*|es kelapa[a-z ]*)","air kelapa",regex=True)
drinkarr=drinkarr.replace(r"random","",regex=True)
drinkarr=drinkarr.replace(r"hi","",regex=True)
drinkarr=drinkarr.replace(r"soda/soft drink","soft drink",regex=True)

In [ ]:
#drinkarr

In [ ]:
drinkdict={}
for i in drinkarr:
    #biar engga duplikat
    temp=dict.fromkeys(i.split(', '))
    if len(temp)<drink_threshold:
      continue
    for j in temp:
        if j in stupidcase:
            continue
        if j not in specialcasedrink:
            if j in drinkdict:
                drinkdict[j]=drinkdict[j]+1
            else:
                drinkdict[j]=1
        else:
            entry=specialcasedrink[j]
            if entry in drinkdict:
                drinkdict[entry]=drinkdict[entry]+1
            else:
                drinkdict[entry]=1

In [ ]:
dict(sorted(drinkdict.items(), key=lambda item: item[1]))

In [ ]:
drink_cat=pd.CategoricalIndex(drinkdict)

In [ ]:
hotencoded_drinkarr=[]
for i in drinkarr:
    temparr=[]
    temp=dict.fromkeys(i.split(', '))
    if len(temp)<drink_threshold:
        hotencoded_drinkarr.append([])
        continue
    for j in drink_cat:
        if j in i.split(', '):
            temparr.append(1)
        else:
            temparr.append(0)
    hotencoded_drinkarr.append(temparr)

In [ ]:
label=[]
rating=[]

counter=0
userarr=[]

for i in range(len(hotencoded_foodarr)):
    if len(hotencoded_foodarr[i])==0 or len(hotencoded_drinkarr[i])==0:
        counter+=1
        continue
    for j in range(len(hotencoded_foodarr[i])):
        label.append(food_cat[j])
        if hotencoded_foodarr[i][j]:
            rate=5
        else:
            rate=1
        rating.append(rate)
    for j in range(len(hotencoded_drinkarr[i])):
        label.append(drink_cat[j])
        if hotencoded_drinkarr[i][j]:
            rate=5
        else:
            rate=1
        rating.append(rate)
    userarr.append(counter)
    counter+=1

In [ ]:
userid=[]
total_cat=len(food_cat)+len(drink_cat)
for i in range(len(label)):
    userid.append(userarr[math.floor(i/total_cat)])

In [ ]:
output=pd.DataFrame([userid,label,rating]).T.rename({0:'userID',1:'movieID',2:'rating'},axis=1)

In [ ]:
output.to_csv('RetrievalV5.csv',index=False)